<div style="display: flex; gap: 10px;">
  <img src="../packages/cadfiles/HOOPS_AI.jpg" style="width: 20%;">
</div>

# HOOPS AI: EXPERIMENTAL - Flow Trainer


In [1]:
import hoops_ai
import os

hoops_ai.set_license(os.getenv("HOOPS_AI_LICENSE"), validate= False)

HOOPS AI version :  1.0.0-dev16 



In [2]:
import pathlib




flows_outputdir = pathlib.Path.cwd().joinpath("out")

## Prepared the ml input files for ml training

this will only create the graph files (.dgl) for training the GraphClassification model

This is optimized for Trianing once you knw what data you need to extract.

In [3]:
# Import task functions from external module for ProcessPoolExecutor compatibility
from cad_tasks_fabwave import gather_fabwave_files, encode_data_for_ml_training, get_flow_name
flow_name = get_flow_name()

print(f"Flow name: {flow_name}")

2026-01-20 22:27:47 | INFO | numexpr.utils | NumExpr defaulting to 16 threads.


HOOPS AI version :  1.0.0-dev16 

Flow name: ETL_Fabwave_training


In [ ]:

# data source
datasources_dir = [str(pathlib.Path.cwd().parent.joinpath("packages","cadfiles","fabwave"))]

# Create and run the Data Flow
cad_flow = hoops_ai.create_flow(
    name=flow_name,
    tasks=[gather_fabwave_files, encode_data_for_ml_training],
    max_workers=4,
    #debug=True,
    flows_outputdir=str(flows_outputdir),
    ml_task="Part Classification",
    storage_type = "memory",
    #parallel_task_kwargs=parallel_task_kwargs,
    export_visualization=False  # Disable visualization export
)

# Run the flow to process all files
print("Starting flow execution with parallel processing...")
flow_output, output_dict, flow_file = cad_flow.process(inputs={'cad_datasources': datasources_dir})

## Define the data split and training conditions

In [4]:
from hoops_ai.dataset import DatasetLoader 
# read the dgl files form the flow output and inject it to the dataset loader
#dgl files
dgl_dir = flows_outputdir.joinpath("flows",flow_name, "dgl")
print(f"DGL files directory: {dgl_dir}")
# get all .ml file in that folder a list
dgl_files = [str(f) for f in dgl_dir.glob("*.ml")]
print(f"Number of DGL files generated: {len(dgl_files)}")


cadflowdataset = DatasetLoader(graph_files=dgl_files, file_code_start=1)

cadflowdataset.split(key="random", train=0.8, validation=0.1, test=0.1)

DGL files directory: C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\flows\ETL_Fabwave_training\dgl
Number of DGL files generated: 4541


(3633, 454, 454)

## Train GraphClassification model

In [5]:
from hoops_ai.ml.EXPERIMENTAL import GraphClassification, FlowTrainer
# Define the Graph Classification model

PartClassification_HOOPS = GraphClassification(num_classes=45,
                                               result_dir = flows_outputdir)

flow_trainer_HOOPS = FlowTrainer(

    flowmodel       = PartClassification_HOOPS,
    datasetLoader   = cadflowdataset,
    experiment_name = "HOOPS_AI_train",
    result_dir      = flows_outputdir,
    accelerator     = 'cpu',
    devices         = 1, #[0]
    max_epochs      = 1,
    batch_size      = 32  # Reduced from 128 to 64 for memory efficiency
)



GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


GRAPH CLASSIFICATION - HOOPS GAT - MODEL


In [6]:
corrupted_items = flow_trainer_HOOPS.purify()


    -----------------------------------------------------------------------------------
    GRAPH CLASSIFICATION - HOOPS GAT - MODEL  - PURIFICATION STEP
    -----------------------------------------------------------------------------------
    Purification batch size           : 1
    Requested processes               : 1
    Adjusted learning rate (for batch): 6.25e-05
    
    Train set contains                : 3633 samples
    Validation set contains           : 454 samples
    Test set contains                 : 454 samples
    Total samples                     : 4541
    -----------------------------------------------------------------------------------
            


Purification Progress:   0%|                                                                  | 0/4541 [00:00<…

In [7]:
print("Removing corrupted items :", len(corrupted_items))

Removing corrupted items : 752


In [8]:
trained_HOOPS_model = flow_trainer_HOOPS.train()
print(f"Training finished. Model checkpoint saved in {trained_HOOPS_model}")
flow_trainer_HOOPS.test(trained_HOOPS_model)
print(f"Testing finished")


-----------------------------------------------------------------------------------
GRAPH CLASSIFICATION - HOOPS GAT - MODEL - TRAINING STEP
-----------------------------------------------------------------------------------
Training batch size               : 32
Adjusted learning rate (for batch): 0.002

Train set contains                : 3022 samples (81.79%)
Validation set contains           : 338 samples (9.15%)
Test set contains                 : 335 samples (9.07%)
Total samples                     : 3695
Max Epoch                         : 1

The trained model: C:\Users\LuisSalazar\MAIN\repos\HOOPS-AI-tutorials\notebooks\out\ml_output\HOOPS_AI_train\0120\222752\best.ckpt

To monitor the logs, run:
tensorboard --logdir results/HOOPS_AI_train/0120/222752
-----------------------------------------------------------------------------------
        


Training: 0it [00:00, ?it/s]

IndexError: Index 3813 is out of range for graph_files (length: 3808). This might indicate a mismatch between dataset indices and actual file lists. Dataset has 3808 file codes.